References to the documentation describing the structure of the JSON objects:  [AlienVault Object](https://otx.alienvault.com/api)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
import os

In [2]:
os.listdir("bronze/alien_vault")

['alien_vault_batch_1.ndjson',
 'alien_vault_batch_3.ndjson',
 'alien_vault_batch_2.ndjson']

In [3]:
# Init Spark
spark = SparkSession.builder.appName("AlienVaultIngest").getOrCreate()

# Load alien vault jsons
df_raw = spark.read.json("bronze/alien_vault/*.ndjson")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/06 12:55:31 WARN Utils: Your hostname, MacBook-Pro-de-Macia.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.138 instead (on interface en0)
25/09/06 12:55:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/06 12:55:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/06 12:55:33 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: bronze/alien_vault/*.ndjson.
java.io.FileNotFoundException: File bronze/alien_vault/*.ndjson does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileS

# Creating the Dataset from AlienVault Response

In this notebook, we are building a structured dataset based on the raw data returned by **AlienVault**.  
The raw data comes in JSON format, where each record contains a `"response"` field with multiple elements. Each element includes information such as the threat `name`, `description`, `TLP` (Traffic Light Protocol), associated `tags`, and `malware_families`.  

We first define a **schema** for the elements inside `"response"` and for the overall JSON structure. Then, we read all the `.ndjson` files from the Bronze layer using this schema.  

After loading the raw data, we **flatten** the nested `"response"` array using the `explode` function and aggregate the fields by `id` and `file_extracted`. This results in a dataset where all information related to each file is collected in lists, making it easier to analyze and process.


In [4]:
from pyspark.sql.functions import from_json, col, explode
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, BooleanType, IntegerType
from pyspark.sql import functions as F


# Schema for the elements inside response
response_element_schema = StructType([
    StructField("name", StringType(), True),
    StructField("TLP", StringType(), True),
    StructField("description", StringType(), True),
    StructField("Tags", ArrayType(StringType()), True),
    StructField("malware_families", ArrayType(StringType()), True)
])

response_schema = StructType([
    StructField("id", StringType(), True),
    StructField("file_extracted", StringType(), True),
    StructField("response", ArrayType(response_element_schema), True)
    ]
)

df_raw = spark.read.schema(response_schema).json("bronze/alien_vault/*.ndjson")

df_flat = (
    df_raw
    .withColumn("response", explode(col("response")))
    .groupBy("id", "file_extracted")
    .agg(
        F.collect_list(col("response.name")).alias("names"),
        F.collect_list(col("response.description")).alias("descriptions"),
        F.collect_list(col("response.TLP")).alias("TLPs"),
        F.collect_list(col("response.tags")).alias("tags"),
        F.collect_list(col("response.malware_families")).alias("malware_families")
    )
)

df_flat.head(2)

25/09/06 12:55:34 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: bronze/alien_vault/*.ndjson.
java.io.FileNotFoundException: File bronze/alien_vault/*.ndjson does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analys

[Row(id='0.client-channel.google.com', file_extracted='/app/ingestion/../data_to_extract/white_list_domain.txt', names=['23.219.89.169  dty-274d7ae9-e5e0-48eb-80db-f8daf26a8d1b-default-prod-64333_23af6e1645db3b350058.js', 'recaptcha__pl.js', 'Cyber Army', 'Project Skynet '], descriptions=['https://www.virustotal.com/gui/file/191475f518a3563c8bbe32e742cc9106c0c968e2e3b9ce12aa12b5f018cbac42/relations\nhttps://www.virustotal.com/gui/ip-address/23.219.89.169/relations', '', '174.bm-nginx-loadbalancer.mgmt.sin1.adnexus.net\nCyber Army, Project Skynet, malware, malware site, Deobfuscate/Decode Files or Information\nAttacking www songcultre.com found in  malicious cloudfront.net link.', ''], TLPs=['white', 'white', 'white', 'white'], tags=[], malware_families=[[], [], [], []]),
 Row(id='17track.net', file_extracted='/app/ingestion/../data_to_extract/white_list_domain.txt', names=['Remote Network Attack | JakyllHyde: Malicious Keyword Tool Index | Sabey Data Centers'], descriptions=["Research 

# Assigning Threat Status Based on File Source

In this step, we are creating a new column called `threat_status` to classify each file based on its source.  

- If the `file_extracted` path contains `"black_list"`, the file is labeled as **"malicious"**.  
- If it contains `"white_list"`, it is labeled as **"whitelist"**.  
- Otherwise, the status is set to **"unknown"**.  

This allows us to quickly categorize files and assess potential threats for further analysis.

In [5]:
from pyspark.sql import functions as F

df_flat = df_flat.withColumn(
    "threat_status",
    F.when(F.col("file_extracted").contains("black_list"), "malicious")
     .when(F.col("file_extracted").contains("white_list"), "whitelist")
     .otherwise("unknown")
)

In [6]:
df_final = df_flat.toPandas()
df_final = df_final.drop('file_extracted', axis=1)
df_final.columns

Index(['id', 'names', 'descriptions', 'TLPs', 'tags', 'malware_families',
       'threat_status'],
      dtype='object')

# Cleaning and Preparing Columns for Analysis

In this step, we are processing and cleaning some of the key columns in `df_final` to prepare the data for further analysis, such as Natural Language Processing (NLP) models.

- **TLPs**: We remove duplicate entries from the `TLPs` lists by converting each list into a dictionary (to keep only unique values) and then back to a list. This results in an array of unique TLP categories for each record.
  
- **names**: Multiple names associated with each file are combined into a single string, joining them with spaces. This makes it easier to feed the data into NLP models.

- **descriptions**: Similarly, multiple descriptions are joined into a single string. Additionally, any URLs (starting with http://, https://, or www.) are excluded to keep only textual information relevant for analysis.

This preprocessing ensures that names and descriptions are ready as clean text inputs for NLP models, while TLPs remains a structured array of categories.

In [7]:
df_final['TLPs'] = df_final['TLPs'].apply(lambda x: list(dict.fromkeys(x)))

df_final["names"] = df_final["names"].apply(lambda x: " ".join(x))

df_final["descriptions"] = df_final["descriptions"].apply(
    lambda x: " ".join([w for w in x if not w.startswith(("http://", "https://", "www."))])
)

As we have seen, `tags` and `malware_families` are always empty, so we can remove them to obtain a cleaner dataset.

In [8]:
df_final = df_final.drop(['tags', 'malware_families'], axis=1)
df_final.head()

,id,names,descriptions,TLPs,threat_status
0,0.client-channel.google.com,23.219.89.169 dty-274d7ae9-e5e0-48eb-80db-f8d...,174.bm-nginx-loadbalancer.mgmt.sin1.adnexus.n...,[white],whitelist
1,17track.net,Remote Network Attack | JakyllHyde: Malicious ...,Research shows compromise originated from Sabe...,[green],whitelist
2,1drv.com,DarkWatchman Chekin Activity Order Brian Sabe...,Brian Sabey & large team continue excessive ...,[green],whitelist
3,25z5g623wpqpdwis.onion.to,IOC Records Provided by @NextRayAI IOCs Indust...,This IOC report provided and daily updated by ...,[white],malicious
4,27lelchgcvs2wpm7.3lhjyx.top,TomkompSerwis 5b685b6fd0c356b8389e33596a40c6...,Dŵr dysku zewnętrznego wedi cymryd i'wodraeth ...,[white],malicious


Ffinally saving the `Silver` dataset for `alien_vault`.

In [9]:
df_final.to_csv('silver/alien_vault/alien_vault.csv', sep=';', index=False)

In [10]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3024 entries, 0 to 3023
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             3024 non-null   object
 1   names          3024 non-null   object
 2   descriptions   3024 non-null   object
 3   TLPs           3024 non-null   object
 4   threat_status  3024 non-null   object
dtypes: object(5)
memory usage: 118.2+ KB
